In [1]:
import pandas as pd

#load data
train_prep = pd.read_csv('./train_prep.csv')
test_prep = pd.read_csv('./test_prep.csv')

train_prep.head(5)

,target,ps_ind_01,ps_ind_03,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,ps_ind_12_bin,...,ps_car_11_cat_oh_50,ps_car_11_cat_oh_4,ps_car_11_cat_oh_58,ps_car_11_cat_oh_9,ps_car_11_cat_oh_17,ps_car_11_cat_oh_11,ps_car_11_cat_oh_45,ps_car_11_cat_oh_14,ps_car_11_cat_oh_81,ps_car_11_cat_oh_47
0,0,2,5,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,7,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,5,9,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,2,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [2]:
%%time

import h2o
from h2o.automl import H2OAutoML

h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.7" 2020-04-14; OpenJDK Runtime Environment (build 11.0.7+10-post-Ubuntu-2ubuntu218.04); OpenJDK 64-Bit Server VM (build 11.0.7+10-post-Ubuntu-2ubuntu218.04, mixed mode, sharing)
  Starting server from /anaconda/envs/py37_default/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp8wn4yugi
  JVM stdout: /tmp/tmp8wn4yugi/h2o_adelsors_started_from_python.out
  JVM stderr: /tmp/tmp8wn4yugi/h2o_adelsors_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,07 secs
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.9
H2O cluster version age:,7 months and 2 days !!!
H2O cluster name:,H2O_from_python_adelsors_su54ii
H2O cluster total nodes:,1
H2O cluster free memory:,27.51 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


CPU times: user 138 ms, sys: 79.1 ms, total: 217 ms
Wall time: 11.5 s


In [3]:
%%time 

#load data as h2o frames
train = h2o.H2OFrame(train_prep)
test = h2o.H2OFrame(test_prep)

#identify predictors and labels
x = train.columns
y = 'target'
x.remove(y)

#for binary classification, labels should be a factor
train[y] = train[y].asfactor()

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
CPU times: user 1min 58s, sys: 6.98 s, total: 2min 5s
Wall time: 2min 13s


In [6]:
%%time 

# Run AutoML 
aml_ti = H2OAutoML(max_runtime_secs = 43200, 
                   #max_models= 20, not limited by default 
                   seed= 7, 
                   nfolds= 10, 
                   balance_classes=True,
                   sort_metric='auc',
                   verbosity='info')

aml_ti.train(x = x, y = y, training_frame = train)
          
#check the leaderboard
lb_ti = aml_ti.leaderboard
lb_ti

AutoML progress: |
17:21:24.335: Project: automl_py_1_sid_87da
17:21:24.335: AutoML job created: 2020.05.31 17:21:24.335
17:21:24.335: Build control seed: 7
17:21:24.336: training frame: Frame key: automl_training_py_1_sid_87da    cols: 208    rows: 595212  chunks: 125    size: 27468258  checksum: 2952209417874460521
17:21:24.336: validation frame: NULL
17:21:24.336: leaderboard frame: NULL
17:21:24.336: blending frame: NULL
17:21:24.336: response column: target
17:21:24.336: fold column: null
17:21:24.336: weights column: null
17:21:24.337: Setting stopping tolerance adaptively based on the training frame: 0.0012961765521830003
17:21:24.339: AutoML build started: 2020.05.31 17:21:24.338

███
17:44:43.296: New leader: XGBoost_1_AutoML_20200531_172124, auc: 0.6382894204463165

████
18:35:27.631: New leader: XGBoost_3_AutoML_20200531_172124, auc: 0.6402781480932414
18:35:27.632: AutoML: starting GLM hyperparameter search

█████████████████████████████████████████████████
21:22:04.296: Au

model_id,auc,logloss,mean_per_class_error,rmse,mse
StackedEnsemble_AllModels_AutoML_20200531_172124,0.641731,0.152167,0.427612,0.186491,0.0347791
XGBoost_grid_1_AutoML_20200531_172124_model_1,0.641673,0.151781,0.43523,0.186354,0.0347277
XGBoost_grid_1_AutoML_20200531_172124_model_6,0.641541,0.151814,0.432735,0.186377,0.0347364
StackedEnsemble_BestOfFamily_AutoML_20200531_172124,0.641109,0.152313,0.432411,0.186549,0.0348004
XGBoost_grid_1_AutoML_20200531_172124_model_3,0.640695,0.151863,0.43389,0.18637,0.0347336
XGBoost_3_AutoML_20200531_172124,0.640278,0.151909,0.443604,0.186382,0.0347384
XGBoost_1_AutoML_20200531_172124,0.638289,0.152087,0.438056,0.18646,0.0347673
GBM_1_AutoML_20200531_172124,0.637438,0.152162,0.428976,0.186476,0.0347734
GBM_grid_1_AutoML_20200531_172124_model_4,0.636867,0.152168,0.434166,0.186469,0.0347707
GBM_grid_1_AutoML_20200531_172124_model_1,0.635721,0.153347,0.433974,0.18679,0.0348905


In [7]:
%%time 

#prediction
pred = aml_ti.leader.predict(test)

stackedensemble prediction progress: |████████████████████████████████████| 100%
CPU times: user 1.09 s, sys: 58.3 ms, total: 1.15 s
Wall time: 3min 24s


In [8]:
pred_df = pred.as_data_frame()
pred_df.head(5)

,predict,p0,p1
0,0,0.971698,0.028302
1,0,0.973806,0.026194
2,0,0.971772,0.028228
3,0,0.979050,0.020950
4,0,0.963408,0.036592


In [10]:
#save predict results to submission form
submit = pd.read_csv("./sample_submission.csv",sep=',')
preds = pd.read_csv("save/target_predictions.csv")

submit['target'] =  pred_df["p1"].values

submit.to_csv("./results-h2o-20200601.csv", index=False)